# Automating the API

Now that we have successfully run one query, we can run automate things. This means we can make work that would otherwise be very bulky much easier. 

Say, for example, we wanted to find out what records were held at TNA for every ship in the Royal Navy during the war of 1812? By hand, this would require a huge amount of browsing. With automation, this can be made a lot easier. 

The first thing we need is a list of ship names. Helpfully, this is [available on Wikipedia](https://en.wikipedia.org/wiki/Category:War_of_1812_ships_of_the_United_Kingdom). The full list is shown in [the aditional data file](./aditional_data.py). Here, we are going to show just the first 3 ships. 
Note that, to simplify the code, we are using the names of ships with an HMS prefix, and only one name. 

`
three_ships = [
    "Devastation",
    "Dreadnought",
    "Invincible"
]
`

In [ ]:
three_ships = [
    "Devastation",
    "Dreadnought",
    "Invincible"
]

With some research, we can see that TNA has provided guides on how to search teh archives. In particular, they have provided information on [how to find logs and records from Royal Navy ships](https://www.nationalarchives.gov.uk/help-with-your-research/research-guides/royal-navy-ships-voyages-log-books/). From this, we can create another list - the various record series we would want to search. 

From the API documentation we saw before, we can see that these record series have to be included in a specific format. As with the list of ship names, we've included the full list in [the aditional data file](./aditional_data.py). Here, we are going to show just 3 record series.

`"sps.recordSeries=ADM%2055"` <- record series ADM 55

`"sps.recordSeries=ADM%20101"` <- record series ADM 101

`"sps.recordSeries=MT%2032"` <- record series MT 32

In [ ]:
"sps.recordSeries=ADM%2055", # record_series_ADM_55
"sps.recordSeries=ADM%20101", # record_series_ADM_101
"sps.recordSeries=MT%2032" # record_series_MT_32

With two lists, we can start to automate the requests. 

We will start by importing the libraries we need, along with the full length lists. 

In [1]:
%pip install requests
%pip install json
%pip install time
import requests
import json
import time

import aditional_data

ship_list = aditional_data.ships

record_series = aditional_data.admiralty_record_series

ship_data = []

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to

As we are going to automate the requests, we are going to need a place to store the results. We are going to use a list of list of dictionaries, making our results both easy to read and easy to work with. 

From the discovery API documentation, we can decide what bits of information we want. We are going to use the following:
- The ship name
- The record start date
- The record end date
- The record ID
- The record description
- Whether the record is digitised

It is often helpful to write an example of the results you want. It helps to visualise your goal, making it easier to build code. An example of the results we are looking for is below. 

```
    [
        {
            "ship-name": "Devastation",
            "records" : [
                {
                    "startDate": "1871-01-01",
                    "endDate": "1871-12-31",
                    "id": "C1234567",
                    "description": "A description of the record",
                    "disgitised": True
                },
                {
                    "startDate": "1872-01-01",
                    "endDate": "1872-12-31",
                    "id": "C1234568",
                    "description": "A description of the record",
                    "disgitised": True
                }
            ]
        }, ## These outer brackets, and all data within, are then repeated for each ship
]
```

In [ ]:
[
    {
        "ship-name": "Devastation",
        "records" : [
            {
                "startDate": "1871-01-01",
                "endDate": "1871-12-31",
                "id": "C1234567",
                "description": "A description of the record",
                "disgitised": True
            },
            {
                "startDate": "1872-01-01",
                "endDate": "1872-12-31",
                "id": "C1234568",
                "description": "A description of the record",
                "disgitised": True
            }
        ]
    }, ## These outer brackets, and all data within, are then repeated for each ship
]

So, now we have a list of ships, a list of record series, and a list of the information we want. We can start to build our code.

This next cell is where where the requests actually happen - hence, there are line-by-line comments to give a detailed explanation of what is happening. The general aim of this cell is to show the power of an API and automation. Here, instead of having to search for each ship by hand, we can loop through the list and search for each ship automatically.

There are two further bits of information we need to note. 
1 - We are going to artificially limit the searches to the first 10 entries in the list. This is to prevent the Discovery API from rejecting our requests. From the [help page](https://www.nationalarchives.gov.uk/help/discovery-for-developers-about-the-application-programming-interface-api/), there is a request for less than 3,000 requests a day, and a rate of approx 1/sec. These limits are often in place on API's to prevent from overloading the servers - if several people were to seperately run all the requests in this notebook, an accidental [DDoS](https://en.wikipedia.org/wiki/Denial-of-service_attack) could occur. This is not going to happen with the requests in this notebook, but it is good practice to limit the number of requests you make, and for servers to limit the number of requests they accept.

Another approach to this would be to slow down the requests - you could wait for a set period of time between each request by using the [time.sleep()](https://docs.python.org/3/library/time.html#time.sleep) function. 

2 - We are going to need to do these requests in two steps. From the documentation, we can see that start and end date, and id are all avaialable from the general search, btu the description and digitisation status are not. To get these, we need to make a second request to the [`/records/details/{id}` endpoint](https://discovery.nationalarchives.gov.uk/API/sandbox/index#/Records). These requests will need to be for individual records, and will provide the extra information we need.

Note: this next cell can take a couple of minutes to run. 

In [ ]:
for ship in ship_list:                                                                  # We start by looping through each ship in the list
        url = "https://discovery.nationalarchives.gov.uk/API/search/records?"           # For every ship, we need to build a custom URL
        for record in record_series:                                                    # We need to add each record series we want to search within, so we loop through the list of record series
                url += record                                                           # We add the record series to the URL
                url += "&"                                                              # An ampersand is added to seperate query parameters
        url += "sps.searchQuery=" + ship                                                              # We add the query parameter to the URL - this is the ship name 
        headers = {
                'Accept': 'application/json'                                            # Setting the headers indicates to the API that we want to receive JSON data. This will make it easier to work with the response data
            }
        response = requests.request("GET", url, headers=headers)                        # THe request is actually made! We store the response in a variable called response
        response_json = response.json()                                                 # We convert the response to JSON format within python.
        if response_json["records"] != []:                                              # In case some ships don't have any records, we need to check that the records list isn't empty
                records_found = []                                                      # Creating the list to store information about each record found
                for record in response_json["records"]:                        
                        records_found.append(                                           # For each record found, we add a dictionary to the list of records found
                                {
                                        "id": record["id"],                             # This dictionary contains the id
                                        "startDate": record["startDate"],               # Start date
                                        "endDate": record["endDate"]                    # And end date of the record - this is all this API endpoint returns
                                }
                        )
                ship_data.append(                                                       # Once we've looped through all the records, we add a dictionary to the list of ship data
                        {
                                "ship-name": ship,                                      # We want the ship name
                                "records": records_found                                # And the list of records found
                        }
                )

print(json.dumps(ship_data, indent=4, sort_keys=True))                                   # After we've looked through all the ship names, print the ship data to check it's worked. Here, we use the json.dumps() function to print the data in a readable format

Knowing whether the record is digitised is going to be valueable. If it is, we can download it from the Discovery GUI - using the API means we can go directly to the records, rather than having to browse. This is also going to be useful for the next notebook, where we are going to visualise various aspects of the records. 

As we have seen in the documentation, we are going to need to do a call to a different endpoint to get the extra information we need. This is going to be a seperate request, and will need to be done for each record. We then pick the information we want from the result, and add it to the data we've already collected for each record.

As we indicated before, we are going to limit the number of requests to avoid getting rejected by the discovery API. Here, rather than limiting the rate of requests, we are simply going to limit to the first 10 ships. The mechanism for doing this will be explained in-line, and will be used again in the notebook where we need to. 

In [ ]:
details_url = "https://discovery.nationalarchives.gov.uk/API/records/v1/details/" 

for ship in ship_data[:10]:             # The [:10] here is a slice - it means we only want to look at the first 10 ships in the list
    for record in ship["records"]:      # For each ship, we want to look at each record. Note that this gives us a double-loop - this is usually a sign that we are going to create a lot of API calls!
        url = details_url + record["id"] # Build the URL for the details endpoint
        headers = {
            'Accept': 'application/json'
        }
        response = requests.request("GET", url, headers=headers)
        response_json = response.json()
        record["description"] = response_json["scopeContent"]["description"] # We can add more data to the dictionary for each record by simply adding a new key and value
        record["digitised"] = response_json["digitised"]                 

print(json.dumps(ship_data[:10], indent=4, sort_keys=True))  # Print the ship data again to check the new data has been added

## Searching within series

There are other endpoints avaiable, allowing us to do more complex and detailed searches. 

### Children within a series

With some investigation of the Discovery website browse section, we can see that major record series are split into subseries. Helpfully for automation, these subseries are numberical (for example, the Admiralty series has the subseries ADM 1, ADM 2, ADM 3, etc). 
Combining this with the `/records/children/{id}`` endpoint, we can search within a series as deep as we want (or untill the API limits are reached). We can use these results to find interesting information about the records, such as finding series which cover a specific date range, or series with a greater number of children (indicative of a larger number of records).

#### Code details

The code for this section is more complicated, using a recursive function to search through the children. This is a function that calls itself, and is a common way of searching through nested data structures. We will take this line by line, and explain what is happening.


### Notes on using this code and the API

The main thing to be aware of is that the number of children can become very large, very fast. If this is the case, it is VERY strongly reccomended that you limit the depth to 3 levels, or 2 if the series is particularly large (such as the Admiralty series). If you do not do this, you will very quickly reach the API limits, and will be unable to make any more requests. 

For most of the children, the id will be a C-code. This can then be used to find the record in the discovery website. If the child has a longer b-code, this is the equivelent - it just means the record is born digital (the code is still doing what it should).

In [13]:
baseurl = "https://discovery.nationalarchives.gov.uk/API"
search_endpoint = "/search/records"
children_endpoint = "/records/children/"

def find_first_level_children(series, max_depth):
    found_children = []
    for i in range(1, 1000):                        # There are not 1000 children of any series, but this is a safe upper limit to use
        time.sleep(1)                               # We add a sleep here to avoid overloading the API
        url = baseurl + search_endpoint + "?sps.recordSeries=" + series + "%20" + str(i) + "&sps.catalogueLevels=Level3" # We build the URL for the search endpoint. We add the catalogueLevels parameter to only return records at level 3 - this is the top level of each series (e.g. ADM)
        headers = {
            'Accept': 'application/json'
        }
        response = requests.request("GET", url, headers=headers)
        response_json = response.json()
        if response_json["records"] != []:
            #print(json.dumps(response_json, indent=4, sort_keys=True))
            found_children.append(
                {
                    "url": url, 
                    "series": series+" "+str(i),
                    "id": response_json["records"][0]["id"],
                    "startDate": response_json["records"][0]["startDate"],
                    "endDate": response_json["records"][0]["endDate"],
                    "children": find_further_children(
                        [
                            {
                                "id": response_json["records"][0]["id"]
                            }
                        ],
                        1,
                        int(max_depth)
                    )
                }
            )
        else:
            break
    return found_children

def find_further_children(series, current_depth, max_depth):
    current_depth += 1
    for individual in series:
        time.sleep(1)                                                   # Sleep here as well, avoiding overloading the API
        url = baseurl + children_endpoint + individual["id"]
        headers = {
            'Accept': 'application/json'
        }
        response = requests.request("GET", url, headers=headers)
        response_json = json.loads(response.text)
        if response.status_code == 200:                                     # Check that the API has returned a 200, as it will return 403 if we have hit the limit of API calls
            if len(response_json["assets"]) > 0:
                assets = []
                for asset in response_json["assets"]:
                    if current_depth < max_depth:
                        children = find_further_children(                   # Here, we call this same funciton again, if we haven't reached the maximum depth, and there are children to find. The return statement below means that this will be populated with the children of the current asset
                            [
                                {
                                    "id": asset["id"]
                                }
                            ],
                            current_depth,                                  # We need to pass the current depth to the function, so it can keep track of how deep it is
                            max_depth                                       # And how deep it is allowed to go. This is the value the user entered at the start. 
                        )
                    else: 
                        children = ["max depth reached"]
                    assets.append(
                        {
                            "id": asset["id"],
                            "coveringFromDate": asset["coveringFromDate"],
                            "coveringToDate": asset["coveringToDate"],
                            "children": children
                        }
                    )
                series = assets
            else:
                assets = ["no assets"]
    return series

series = input("Enter the series you want to search within: ")
max_depth = input("Enter the maximum depth you want to search to: ")
output = {}
output["top_level"] = series
output["children"] = find_first_level_children(series, max_depth)
print(json.dumps(output, indent=4, sort_keys=False))                        # Print the output to check it's worked. We could do other things, such as write it to a file, or pass it to another function


{
    "top_level": "ACE",
    "children": [
        {
            "url": "https://discovery.nationalarchives.gov.uk/API/search/records?sps.recordSeries=ACE%201&sps.catalogueLevels=Level3",
            "series": "ACE 1",
            "id": "C13487293",
            "startDate": "01/01/2004",
            "endDate": "31/12/2012",
            "children": [
                {
                    "id": "b27b0db0584d4b159036ffbcf47a5b05",
                    "coveringFromDate": 20121231,
                    "coveringToDate": 20121231,
                    "children": [
                        {
                            "id": "b27b0db0584d4b159036ffbcf47a5b05"
                        }
                    ]
                },
                {
                    "id": "e1d86956b53c4014ae2b3650cc10131f",
                    "coveringFromDate": 20121231,
                    "coveringToDate": 20121231,
                    "children": [
                        {
                            "id": "